In [1]:
import cv2 as cv
import numpy as np



In [2]:
from IPython.display import display,Image

In [3]:
# pip install roboflow


In [4]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="skGiO46ei6WKyfpYHdrh")
# project = rf.workspace("hackathon-project").project("test-7xped")
# dataset = project.version(1).download("yolov8")

In [5]:
#pip install ultralytics --user

In [6]:

# from ultralytics import YOLO

In [7]:
# model=YOLO('yolov8n.pt')
# results=model.predict(source='test_img.jpg', conf=0.25)

In [8]:
# results2=model.predict(source='standing.jpg', conf=0.25)

In [9]:
pip install opencv-contrib-python

In [38]:
import cv2

# Load the image
img = cv2.imread('test_img.jpg')

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply a Gaussian blur
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# Detect edges
edges = cv2.Canny(blur, 0, 400)

# Find contours
contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through contours and find forearm contour
forearm_contour = None
for contour in contours:
    # Calculate centroid of contour
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
        # Check if centroid is in forearm region
        if cx > 100 and cx < 400 and cy > 200 and cy < 500:
            forearm_contour = contour
            break

# Draw forearm contour
cv2.drawContours(img, [forearm_contour], 0, (0, 0, 255), 2)

# Display the image
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
### draw lines on 

In [37]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('test_img.jpg')

# Define the start and end points of the line on the forearm
start_point = (90,200)
end_point = (200, 100)

# Draw the line on the forearm
cv2.line(img, start_point, end_point, (0, 255, 0), thickness=2)

# Display the image with the line on the forearm
cv2.imshow('image with line on forearm', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [40]:
pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25870 sha256=a84b7eefc78a6b15d380f383ec89b976a88670df44c81d167951174dad50dc37
  Stored in directory: c:\users\tukuna\appdata\local\pip\cache\wheels\59\1b\52\0dea905f8278d5514dc4d0be5e251967f8681670cadd3dca89
Successfully built imutils



In [43]:
## from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True, help="test_img.jpg")
ap.add_argument("-w", "--width", type=float, required=True, help="7")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -i IMAGE -w WIDTH
ipykernel_launcher.py: error: the following arguments are required: -i/--image, -w/--width


SystemExit: 2

In [44]:
import cv2
import numpy as np
image = cv2.imread('test_img.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, 50, 150)
lines = cv2.HoughLines(edges, 1, np.pi/180, 200)
abdomen = None
chest = None
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    slope = (y2-y1)/(x2-x1)
    if slope > 0.5:
        abdomen = (x1, y1, x2, y2)
    elif slope < -0.5:
        chest = (x1, y1, x2, y2)
if abdomen is not None:
    cv2.rectangle(image, (abdomen[0], abdomen[1]), (abdomen[2], abdomen[3]), (0, 255, 0), 2)
if chest is not None:
    cv2.rectangle(image, (chest[0], chest[1]), (chest[2], chest[3]), (0, 0, 255), 2)


TypeError: 'NoneType' object is not iterable

In [9]:
import cv2
import numpy as np


webcam=True
path='test_img.jpg'

cap=cv2.VideoCapture(0)
cap.set(10,160)
cap.set(3,1920)
cap.set(4,1080)

wP=210
hP=297
scale=3


def getContours(img, cThr=[100,100], showCanny=False, minArea=1000, filter=0, draw=False):
    imgGray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (5,5), 1)
    imgCanny= cv2.Canny(imgBlur, cThr[0],cThr[1])
    kernel=np.ones((5,5))
    imgDial = cv2.dilate(imgCanny,kernel,iterations=3)
    imgThre= cv2.erode(imgDial,kernel,iterations=2)
    if showCanny:
        cv2.imshow('canny', imgCanny)
    
    contours, hierarchy=cv2.findContours(imgThre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    finalContours=[]
    for i in contours:
        area=cv2.contourArea(i)
        if area> minArea:
            peri=cv2.arcLength(i,True)
            approx=cv2.approxPolyDP(i,0.02*peri,True)
            bbox=cv2.boundingRect(approx)
            if filter>0:
                if len(approx) == filter:
                    finalContours.append([len(approx),area,approx, bbox,i])
            else:
                finalContours.append([len(approx),area,approx, bbox,i])
    finalContours=sorted(finalContours,key=lambda x:x[1], reverse=True)    
    if draw:
        for con in finalContours:
            cv2.drawContours(img,con[4],-1,(0,0,255),3)
    
    return img, finalContours

def reorder(mypoints):
    print(mypoints.shape)
    mypointsnew=np.zeroes_Like(mypoints)
    mypoints=mypoints.reshape((4,2))
    add=mypoints.sum(1)
    mypointsnew[0]=mypoints[np.argmin(add)]
    mypointsnew[3]=mypoints[np.argmax(add)]
    diff=np.diff(mypoints, axis=1)
    mypointsnew[1]=mypoints[np.argmin(diff)]
    mypointsnew[2]=mypoints[np.argmax(diff)]
    
    return mynewpoints

def warpImg(img,points, w,h):
#     print(points)
    points=reorder(points)
    
    pts1=np.float32(points)
    pts2=np.float32([[0,0], [w,0], [0,h],[w,h]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgWrap=cv2.warpPerspective(img,matrix,(w,h))
    
    return imgWarp

    
    
while True:
    if webcam:
        success,img=cap.read()
        img = cv2.flip(img,1)
        cv2.imshow('Original', img)
        if cv2.waitKey(90) & 0xFF==ord('s'): #press s to close the window
            break
        
    
    else:
        img=cv2.imread(path)
        cv2.imshow('Original', img)
        if cv2.waitKey(90) & 0xFF==ord('s'): #press s to close the window
            break
    
    img, conts=getContours(img,showCanny=True, minArea=50000, filter=4)
    
    if len(conts)!=0:
        biggest=conts[0][2]
#         print(biggest)
        imgWarp=warpImg(img,biggest,wP,hP)
        
        imgcontours2, conts2=getContours(imgWarp,showCanny=True, minArea=2000, filter=4, cThr=[50,50], draw=True)
        cv2.imshow('A4',imgcontours2)
    img=cv2.resize(img,(0,0), None, 0.5,0.5) 
    cv2.imshow('Original', img)

    
cap.release()
cv2.destroyAllWindows() 
cv2.waitKey(0)

-1